In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

## Data Preparation

In [2]:
df = pd.read_csv("Housing.csv")

In [3]:
# retrieve labels from https://usa.ipums.org/usa-action/variables/live_search for exploration purposes
col_labels = {
    "SERIAL": "Household serial number",
    "DENSITY": "Population-weighted density of PUMA",  # PUMA is Public Use Microdata Area
    "OWNERSHP": "Ownership of dwelling (tenure) [general version]",  # owned/being bought vs rented
    "OWNERSHPD": "Ownership of dwelling (tenure) [detailed version]",  # e.g. rented vs no cash rent vs cash rent
    "COSTELEC": "Annual electricity cost",
    "COSTGAS": "Annual gas cost",
    "COSTWATR": "Annual water cost",
    "COSTFUEL": "Annual home heating fuel cost",
    "HHINCOME": "Total household income",
    "VALUEH": "House value",
    "ROOMS": "Number of rooms",
    "BUILTYR2": "Age of structure, decade",
    "BEDROOMS": "Number of bedrooms",
    "VEHICLES": "Vehicles available",
    "NFAMS": "Number of families in household",
    "NCOUPLES": "Number of couples in household",
    "PERNUM": "Person number in sample unit",
    "PERWT": "Person weight",  # sampling weight, indicates how many persons in the U.S. population are represented by a given person
    "AGE": "Age",
    "MARST": "Marital status",
    "BIRTHYR": "Year of birth",
    "EDUC": "Educational attainment [general version]",
    "EDUCD": "Educational attainment [detailed version]",
    "INCTOT": "Total personal income",
}

df_readable = df.rename(columns=col_labels)

In [4]:
# filter by columns of interest and replace special missing values with NaN
cols_of_interest = [
    "Household serial number",
    "Population-weighted density of PUMA",
    "Ownership of dwelling (tenure) [general version]",
    "Number of rooms",
    "Age of structure, decade",
    "Number of bedrooms",
    "Number of families in household",
    "Number of couples in household",
    "Age",
    "Marital status",
    "Educational attainment [general version]",
    "Total personal income",
]

missing_values_per_col = {
    "Total personal income": [9999999],
}

df_filtered = df_readable[cols_of_interest].replace(missing_values_per_col, np.nan)

In [5]:
# rename some long column names
cols_rename_map = {
    "Population-weighted density of PUMA": "Population-weighted density",
    "Ownership of dwelling (tenure) [general version]": "Ownership of dwelling",
    "Educational attainment [general version]": "Educational attainment",
}

df_filtered = df_filtered.rename(columns=cols_rename_map)

In [6]:
# filter to single family households with 0 or 1 couples -- 2 or 3 couples of the same family living in the same household is an outlier.
df_single_family = df_filtered[
    (df_filtered["Number of families in household"] == 1)
    & (
        (df_filtered["Number of couples in household"] == 1)
        | (df_filtered["Number of couples in household"] == 0)
    )
]

# remove all rows with missing values
initial_length = len(df_single_family)
df_single_family = df_single_family.dropna()
new_length = len(df_single_family)
rows_dropped = initial_length - new_length
print(f"Removed {rows_dropped} rows with missing values out of {initial_length} ({(rows_dropped / initial_length)*100:.2f}%)")

# aggregate per head of household, defined as a person 18 or older with highest personal income
index = (
    df_single_family[df_single_family["Age"] >= 18]
    .groupby("Household serial number")["Total personal income"]
    .idxmax()
)
df_single_family = df_single_family.loc[index]

# we can also aggregate by sorting, but seems like a bad approach
# df_single_family = df_single_family.sort_values("Total personal income", ascending=False).drop_duplicates("Household serial number")

# drop Number of families in household since it's always 1
# drop Household serial number since we've already aggregated, and it's not useful as a predictor
df_single_family = df_single_family.drop(columns=["Number of families in household", "Household serial number"])

Removed 11840 rows with missing values out of 68131 (17.38%)


In [7]:
# group education into 5 categories: less than high school, high school, some college, bachelor's, graduate
# by default is (], left-exclusive and right-inclusive
bins = [0, 5, 6, 9, 10, 11]
labels = [
    "Less than high school",
    "High school",
    "Some college",
    "Bachelor's",
    "Graduate",
]
df_single_family["Educational attainment"] = pd.cut(
    df_single_family["Educational attainment"],
    bins=bins,
    labels=labels,
    include_lowest=True,  # so 0 is included in 0-5
)

In [8]:
# group marital status into 3 categories: married (spouse present/absent), previously married (separated/divorced/widowed), never married
bins = [1, 2, 5, 6]
labels = [
    "Married",
    "Previously married",
    "Never married",
]
df_single_family["Marital status"] = pd.cut(
    df_single_family["Marital status"],
    bins=bins,
    labels=labels,
    include_lowest=True,
)

In [9]:
# rename ownership of dwelling categories, 1 is Owned or being bought, 2 is Rented
df_single_family = df_single_family.replace(
    {"Ownership of dwelling": {1: "Owned", 2: "Rented"}}
)

In [10]:
# encode categorical predictors as nominal, the distance betweeen our categories is hard to quantify and may not be equal
# e.g. would distance between "Less than high school" and "High school" be the same as "High school" and "Some college"?
nominal_cols = [
    "Educational attainment",
    "Marital status",
]

# one-hot encode (which is different to dummy encode) - doesn't drop first category, easier to interpret and SVM can handle it
df_onehot = pd.get_dummies(df_single_family[nominal_cols], drop_first=False)
df_single_family = pd.concat([df_single_family, df_onehot], axis=1)
df_single_family = df_single_family.drop(columns=nominal_cols)

In [11]:
# group age into 3 categories: 18-34 (young adults), 35-64 (middle-aged adults), 65+ (older adults)
bins = [18, 34, 64, float('inf')]
labels = ["Young adults", "Middle-aged adults", "Older adults"]
# keep age as original data for later
age_groups = pd.cut(
    df_single_family["Age"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

# create 3 separate dataframes for each age group
df_young_adults = df_single_family[age_groups == "Young adults"]
df_middle_aged_adults = df_single_family[age_groups == "Middle-aged adults"]
df_older_adults = df_single_family[age_groups == "Older adults"]

In [12]:
# scaler for later use 
scaler = StandardScaler()

## Modeling

### Young Adults

In [13]:
X = df_young_adults.drop(columns=['Ownership of dwelling'])
y = df_young_adults['Ownership of dwelling']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Radial

In [14]:
svm_rbf = SVC(kernel="rbf", tol=0.1)

kFold = KFold(n_splits=5, shuffle=True, random_state=1)
params = {"C": [0.001, 0.01, 0.1, 1], "gamma": [0.0001, 0.001, 0.01, 1, 10, 100]}
cv = GridSearchCV(
    svm_rbf,
    param_grid=params,
    cv=kFold,
    refit=True,
    n_jobs=-1,
    scoring="accuracy",
    verbose=1,
)
cv.fit(X_train_scaled, y_train)

train_accuracy = cv.best_estimator_.score(X_train_scaled, y_train)
train_error = 1 - train_accuracy
test_accuracy = cv.best_estimator_.score(X_test_scaled, y_test)
test_error = 1 - test_accuracy

best_C_rbf = cv.best_params_["C"]
best_gamma = cv.best_params_["gamma"]
print(f"Radial CV results: C={best_C_rbf}, gamma={best_gamma}")
print("Train error:", train_error)
print("Test error:", test_error)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Radial CV results: C=1, gamma=0.01
Train error: 0.2017412188531973
Test error: 0.20028011204481788


#### Poly

In [16]:
svm_poly = SVC(kernel="poly", tol=0.1)

kFold = KFold(n_splits=5, shuffle=True, random_state=1)
params = {
    "C": [0.001, 0.01, 0.1, 1],
    "degree": np.arange(2, 11, 1),
    "coef0": np.arange(0, 11, 1),
}
cv = GridSearchCV(
    svm_poly,
    param_grid=params,
    cv=kFold,
    refit=True,
    n_jobs=-1,
    scoring="accuracy",
    verbose=1,
)
cv.fit(X_train_scaled, y_train)

train_accuracy = cv.best_estimator_.score(X_train_scaled, y_train)
train_error = 1 - train_accuracy
test_accuracy = cv.best_estimator_.score(X_test_scaled, y_test)
test_error = 1 - test_accuracy

best_C_poly = cv.best_params_["C"]
best_degree = cv.best_params_["degree"]
best_coef0 = cv.best_params_["coef0"]

print(f"Poly CV results: C={best_C_poly}, degree={best_degree}, coef0={best_coef0}")
print("Train error:", train_error)
print("Test error:", test_error)

Fitting 5 folds for each of 396 candidates, totalling 1980 fits


#### Linear

In [ ]:
svm_linear = SVC(kernel="linear", tol=0.1) 

kFold = KFold(n_splits=5, shuffle=True, random_state=1)
params = {"C": [0.001, 0.01, 0.1, 1]}
cv = GridSearchCV(
    svm_linear,
    param_grid=params,
    cv=kFold,
    refit=True,
    n_jobs=-1,
    scoring="accuracy",
    verbose=1,
)
cv.fit(X_train_scaled, y_train)

train_accuracy = cv.best_estimator_.score(X_train_scaled, y_train)
train_error = 1 - train_accuracy
test_accuracy = cv.best_estimator_.score(X_test_scaled, y_test)
test_error = 1 - test_accuracy

best_C_linear = cv.best_params_["C"]

print(f"Linear CV results: C={best_C_linear}")
print("Train error:", train_error)
print("Test error:", test_error)

### Middle-Aged Adults

In [ ]:
X = df_middle_aged_adults.drop(columns=['Ownership of dwelling'])
y = df_middle_aged_adults['Ownership of dwelling']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

#### Radial

In [ ]:
svm_rbf = SVC(kernel="rbf", tol=0.1)

kFold = KFold(n_splits=5, shuffle=True, random_state=1)
params = {"C": [0.001, 0.01, 0.1, 1, 10], "gamma": [0.0001, 0.001, 0.01, 1, 10, 100]}
cv = GridSearchCV(
    svm_rbf,
    param_grid=params,
    cv=kFold,
    refit=True,
    n_jobs=-1,
    scoring="accuracy",
    verbose=1,
)
cv.fit(X_train, y_train)

train_accuracy = cv.best_estimator_.score(X_train, y_train)
train_error = 1 - train_accuracy
test_accuracy = cv.best_estimator_.score(X_test, y_test)
test_error = 1 - test_accuracy

best_C_rbf = cv.best_params_["C"]
best_gamma = cv.best_params_["gamma"]
print(f"Radial CV results: C={best_C_rbf}, gamma={best_gamma}")
print("Train error:", train_error)
print("Test error:", test_error)

#### Poly

In [ ]:
svm_poly = SVC(kernel="poly", tol=0.1)

kFold = KFold(n_splits=5, shuffle=True, random_state=1)
params = {
    "C": [0.001, 0.01, 0.1, 1, 10],
    "degree": np.arange(2, 6, 1),
    "coef0": np.arange(0, 11, 1),
}
cv = GridSearchCV(
    svm_poly,
    param_grid=params,
    cv=kFold,
    refit=True,
    n_jobs=-1,
    scoring="accuracy",
    verbose=1,
)
cv.fit(X_train, y_train)

train_accuracy = cv.best_estimator_.score(X_train, y_train)
train_error = 1 - train_accuracy
test_accuracy = cv.best_estimator_.score(X_test, y_test)
test_error = 1 - test_accuracy

best_C_poly = cv.best_params_["C"]
best_degree = cv.best_params_["degree"]
best_coef0 = cv.best_params_["coef0"]

print(f"Poly CV results: C={best_C_poly}, degree={best_degree}, coef0={best_coef0}")
print("Train error:", train_error)
print("Test error:", test_error)

#### Linear

In [ ]:
svm_linear = SVC(kernel="linear", tol=0.1)  # default is 0.001 but it's very slow

kFold = KFold(n_splits=5, shuffle=True, random_state=1)
params = {"C": [0.001, 0.01, 0.1, 1, 10]}
cv = GridSearchCV(
    svm_linear,
    param_grid=params,
    cv=kFold,
    refit=True,
    n_jobs=-1,
    scoring="accuracy",
    verbose=1,
)
cv.fit(X_train, y_train)

train_accuracy = cv.best_estimator_.score(X_train, y_train)
train_error = 1 - train_accuracy
test_accuracy = cv.best_estimator_.score(X_test, y_test)
test_error = 1 - test_accuracy

best_C_linear = cv.best_params_["C"]

print(f"Linear CV results: C={best_C_linear}")
print("Train error:", train_error)
print("Test error:", test_error)